In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import ktrain
from ktrain import text
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
import re
import string
from nltk.tokenize import RegexpTokenizer

In [29]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 365.7 kB/s eta 0:00:0000:0100:01


In [55]:
cols = ['label','tweet_text']
positive = pd.read_csv('Arabic_tweets_positive.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
positive.head()

,label,tweet_text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [56]:
cols = ['label','tweet_text']
negative = pd.read_csv('Arabic_tweets_negative.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
negative.head()

,label,tweet_text
0,neg,انمي Family Guy الموسم الحلقة السادسة مترجمة H...
1,neg,صدااع من الغباء لي ساعه افكر وش اقول بس غبائها...
2,neg,قتيلا وجريحا… حصيلة تفرقة المتظاهرين في كربلاء...
3,neg,اللهم من أصلح شأنهم وجمع كلمتهم ووحد صفهم ياقا...
4,neg,كوميديا.. الذهبية تذهب بالخطأ لصاحبة المركز ال...


In [57]:
final_data = pd.concat([positive, negative], axis=0)
final_data.head()

,label,tweet_text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [58]:
from sklearn.utils import shuffle
final_data = shuffle(final_data).reset_index()
final_data.head()

,index,label,tweet_text
0,75727,pos,: شااركو 😍😍
1,67330,neg,: 📍تحدي 📍 🔐 ريتويت 🔐 رد على هذه التغريدة مع من...
2,46104,pos,جمالجج🥵 — ❤️
3,51371,neg,دشيي حسابيي كل ساعه انزلل 😡
4,25175,neg,اكيد بس وضع سياسي والاقتصادي منهار صار 😔


In [59]:
final_data= final_data.drop('index',axis=1)
final_data.head()

,label,tweet_text
0,pos,: شااركو 😍😍
1,neg,: 📍تحدي 📍 🔐 ريتويت 🔐 رد على هذه التغريدة مع من...
2,pos,جمالجج🥵 — ❤️
3,neg,دشيي حسابيي كل ساعه انزلل 😡
4,neg,اكيد بس وضع سياسي والاقتصادي منهار صار 😔


In [60]:
final_data.isnull().any(axis=0)

label         False
tweet_text    False
dtype: bool

In [61]:
for letter in '#.][!XR':
    final_data['tweet_text'] = final_data['tweet_text'].astype(str).str.replace(letter,'')

In [62]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [63]:
def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", str(text),  flags=re.UNICODE).split())

In [64]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [65]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [66]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [67]:
def processPost(tweet): 

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    #tweet=remove_repeating_char(tweet)


    # remove emoji
    tweet=remove_emoji(tweet)
    
    return tweet

In [68]:
final_data["tweet_text"] = final_data['tweet_text'].apply(lambda x: processPost(x))

In [69]:
final_data["tweet_text"] = final_data['tweet_text'].apply(remove_non_arabic)

In [70]:
final_data.head()

,label,tweet_text
0,pos,شااركو
1,neg,تحدي ريتويت رد علي هذه التغريده مع منشن ل بانق...
2,pos,جمالجج
3,neg,دشيي حسابيي كل ساعه انزلل
4,neg,اكيد بس وضع سياسي والاقتصادي منهار صار


In [71]:
final_data["tweet_text"] = final_data['tweet_text'].apply(lambda x:remove_repeating_char(x))

In [31]:
#tokenizer = RegexpTokenizer(r'\w+')
#final_data["tweet_text"] = final_data["tweet_text"].apply(tokenizer.tokenize)

In [72]:
final_data["tweet_text"].sample(5)

85367                           راق لي طبعك و انا احب الصعب
37815     احبه واخاف منه هالبحر لكن اتمنا امارس المشي به...
145964                                 ده كده عظمه فشخ واله
134552                        ء لنا في اله ظن لا يخيب ء ء ء
94315     اجبروا الخواطر وراعوا المشاعر وانتقوا كلماتكم ...
Name: tweet_text, dtype: object

In [73]:
df_train = final_data.sample(frac=0.80,random_state=200)

df_train.shape

(118662, 2)

In [74]:
df_test =  final_data.drop(df_train.index)

df_test.shape

(29666, 2)

In [75]:
#X = final_data['tweet_text'].values
#y=final_data['label'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [83]:
# set hyperparameters
maxlen = 64
batch_size = 16
lr = 2e-5
epochs = 3

In [77]:
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'
t = text.Transformer(MODEL_NAME, maxlen=maxlen)

In [80]:
trn = t.preprocess_train(df_train.tweet_text.values, df_train.label.values)
tst = t.preprocess_test(df_test.tweet_text.values, df_test.label.values)

preprocessing train...
language: ar
train sequence lengths:
	mean : 11
	95percentile : 22
	99percentile : 25


/opt/conda/lib/python3.9/site-packages/ktrain/utils.py:625: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 11
	95percentile : 22
	99percentile : 25


In [85]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=batch_size)

In [86]:
history = learner.fit_onecycle(lr, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
7417/7417 [==============================] - 16417s 2s/step - loss: 0.4152 - accuracy: 0.8020 - val_loss: 0.3409 - val_accuracy: 0.8463
Epoch 2/3
7417/7417 [==============================] - 16369s 2s/step - loss: 0.3053 - accuracy: 0.8638 - val_loss: 0.2983 - val_accuracy: 0.8661
Epoch 3/3
7417/7417 [==============================] - 16333s 2s/step - loss: 0.1965 - accuracy: 0.9160 - val_loss: 0.3280 - val_accuracy: 0.8706


In [87]:
learner.validate(val_data=tst)

              precision    recall  f1-score   support

           0       0.85      0.89      0.87     14409
           1       0.89      0.85      0.87     15257

    accuracy                           0.87     29666
   macro avg       0.87      0.87      0.87     29666
weighted avg       0.87      0.87      0.87     29666



array([[12849,  1560],
       [ 2280, 12977]])

In [88]:
p = ktrain.get_predictor(learner.model, t)

In [100]:
p.predict("الهلال افضل فريق كورة")

'pos'

In [101]:
ktrain.load_predictor

<function ktrain.core.load_predictor(fpath, batch_size=32, custom_objects=None)>

In [102]:
p.save('ar-bert-model-dataset1')